In [32]:
import pandas as pd

In [33]:
df = pd.read_csv('zawiadomienia.csv', encoding='utf-8-sig', sep=";")

df['Data zawiadom.'] = pd.to_datetime(df['Data zawiadom.'], format='%d.%m.%Y', errors='coerce')

df = df.dropna(subset=["Oznaczenie", "Data zawiadom."])

df["Data zawiadom."] = pd.to_datetime(df["Data zawiadom."], errors="coerce")

df["month"] = df["Data zawiadom."].dt.month
df["year_month"] = df["Data zawiadom."].dt.to_period("M").astype(str)

df.to_csv("clean_zawiadomienia.csv", index=False, encoding='utf-8-sig')

df = df[(df["Rodzaj zawiad."] == "1P") | (df["Rodzaj zawiad."] == "PM")]
df

,Rodzaj zawiad.,Zawiadomienie,Utworz. o godz.,Urządzenie,Wpływ,Lokaliz. funkc.,Przestój,MPK,Pocz. zakłóc.,Koniec zakłóc.,Oznaczenie,Oznaczenie.1,Data zawiadom.,month,year_month
0,1P,12708097,23:40:28,10217097.0,1.0,PLPA-PR-U12-010801-002,NaN,506181.0,23:39:42,02:54:00,1-Zespół pomp próżniowych BUSCH,KINDER CARDS MODELATOR,2025-06-30,6,2025-06
1,1P,12707957,20:28:55,10179666.0,3.0,PLPA-PR-U32-012110,X,506310.0,20:27:53,00:00:00,1-Trans. Sandwick za walcami 64M,Prep2 Krem KB,2025-06-30,6,2025-06
2,1P,12707755,23:35:47,10166601.0,3.0,PLPA-PR-U32-012210,X,506400.0,23:35:46,00:00:00,1-Zbiornik olejowy 1m^3,Jasny krem (TAV/K CARDS/KJ),2025-06-30,6,2025-06
3,1P,12707745,17:59:02,10101884.0,3.0,PLPA-PR-U43-011445-001,X,506370.0,17:59:01,00:00:00,1-Piec wafla górnego KB2,KB 2. Piece z nawilżaczami,2025-06-30,6,2025-06
7,1P,12707707,16:49:56,10174772.0,3.0,PLPA-PR-U32-012011-001,X,506011.0,16:49:52,00:00:00,1-Silos Cukier S-3 Nowy,Preparacja kremu KJ/KB/KBW,2025-06-30,6,2025-06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
160648,PM,10143679,17:17:55,10080123.0,NaN,PLPA-PR-001-010325-001,X,506225.0,14:13:50,14:20:17,2-Kinder Joy lini-Nalewarka goccia,KJ 1 Modelator,2014-04-01,4,2014-04
160649,PM,10143678,17:12:51,10080120.0,NaN,PLPA-PR-001-010325-001,X,506225.0,14:13:50,14:20:17,1-Kinder Joy lini-Modelator do K Joya,KJ 1 Modelator,2014-04-01,4,2014-04
160650,PM,10143666,11:22:15,10081046.0,NaN,PLPA-PR-011-010174-001,NaN,506368.0,11:22:15,00:00:00,1-Linia pakowania IFCR,IFCR Pakowanie wiewiórka/bałwan/jajko,2014-04-01,4,2014-04
160652,PM,10143508,16:05:51,10081498.0,NaN,PLPA-PR-021-012119-001,NaN,506319.0,16:05:50,00:00:00,1-Tanks,KB Rework,2014-04-01,4,2014-04


In [34]:
import pandas as pd, plotly.graph_objects as go

metric = (
    df.groupby(["Oznaczenie.1", "year_month"])
      .size().rename("Count")
      .reset_index()
)

all_lines = sorted(metric["Oznaczenie.1"].unique())

traces = []
for line in all_lines:
    sub = metric.loc[metric["Oznaczenie.1"] == line]
    traces.append(
        go.Bar(
            x=sub["year_month"],
            y=sub["Count"],
            name=line,
            visible=False
        )
    )
traces[0].visible = True   # any default

# dropdown buttons
buttons = []
for k, line in enumerate(all_lines):
    vis = [False]*len(traces)
    vis[k] = True
    buttons.append(
        dict(
            label=line,
            method="update",
            args=[{"visible": vis},
                  {"title": f"Liczba zgłoszeń – {line}"}]
        )
    )

# final figure
fig = go.Figure(
    data=traces,
    layout=dict(
        title=f"Liczba zgłoszeń – {all_lines[0]}",
        xaxis_title="Rok-miesiąc",
        yaxis_title="Liczba zgłoszeń",
        updatemenus=[dict(
            buttons=buttons,
            direction="down",
            showactive=True,
            x=1.02, y=1.0,
            xanchor="left"
        )]
    )
)
fig.update_layout(barmode="group", height=450, width=900)
fig.show()
